Type of regression: Logistic regression
Purpose of regression: Classification algorithm that predicts a binary outcome (i.e., dependent categorical variable)

Steps in analysis:
1. Chi-square test: Identify individual predictors associated with diabetes and determine whether there are gender differences in frequencies.
2. Overall logistic regression: Evaluate the independent effect of each predictor on diabetes risk while controlling for other factors. This shows which predictors are significant once confounding is accounted for.
3. Logistic regression with interaction terms: Using the identified significant predictors from the overall logistic regression results, test whether the effect of each significant predictor differs by gender. Significant interaction means that the predictor affects men and women differently.
4. Stratified logistic regression by gender: Quantify predictor effects for men and women separately by running two models. 


In [73]:
# Import all relevant libraries to be used in this analysis
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency

In [74]:
# Set random seed

np.random.seed(123)

# Load in dataset
data = pd.read_csv("diabetes_data_encoded.csv")

print(data.head())

   age  gender  polyuria  polydipsia  sudden_weight_loss  weakness  \
0   40       1         0           1                   0         1   
1   58       1         0           0                   0         1   
2   41       1         1           0                   0         1   
3   45       1         0           0                   1         1   
4   60       1         1           1                   1         1   

   polyphagia  genital_thrush  visual_blurring  itching  irritability  \
0           0               0                0        1             0   
1           0               0                1        0             0   
2           1               0                0        1             0   
3           1               1                0        1             0   
4           1               0                1        1             1   

   delayed_healing  partial_paresis  muscle_stiffness  alopecia  obesity  \
0                1                0                 1         1 

In [77]:
# Create contingency table
contingency_table = pd.crosstab(data['gender'], data['diabetes'])
print(contingency_table)

# Run Chi-squared test between gender and diabetes
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-squared Statistic:", chi2)
print("Degrees of Freedom:", dof)
print("P-value:", p)

diabetes    0    1
gender            
0          19  173
1         181  147
Chi-squared Statistic: 103.03685927972559
Degrees of Freedom: 1
P-value: 3.289703730553294e-24


In [78]:
# Creating loop for all variables
categorical_vars = [
    'polyuria', 
    'polydipsia', 
    'sudden_weight_loss', 
    'weakness', 
    'polyphagia', 
    'genital_thrush', 
    'visual_blurring', 
    'itching', 
    'irritability', 
    'delayed_healing', 
    'partial_paresis', 
    'muscle_stiffness', 
    'alopecia', 
    'obesity'
    ]

results = []

for var in categorical_vars: 
    table = pd.crosstab(data[var], data['diabetes'])
    chi2, p, dof, expected = chi2_contingency(table)
    results.append({'Variable': var, 'Chi2': chi2, 'p-value': p})

results_data = pd.DataFrame(results).sort_values('p-value')
print(results_data)

              Variable        Chi2       p-value
0             polyuria  227.865839  1.740912e-51
1           polydipsia  216.171633  6.187010e-49
2   sudden_weight_loss   97.296303  5.969166e-23
10     partial_paresis   95.387627  1.565289e-22
4           polyphagia   59.595254  1.165158e-14
8         irritability   45.208348  1.771483e-11
12            alopecia   36.064143  1.909279e-09
6      visual_blurring   31.808456  1.701504e-08
3             weakness   29.767918  4.869843e-08
11    muscle_stiffness    7.288667  6.939096e-03
5       genital_thrush    5.792149  1.609790e-02
13             obesity    2.327474  1.271080e-01
9      delayed_healing    0.962094  3.266599e-01
7              itching    0.046235  8.297484e-01


The chi-square test shows that polyuria, polydipsia, sudden weight loss, and partial paresis have the stongest associations with diabetes (individual correlation with diabetes).

In [ ]:
## [EITHER SCALE AGE OR BIN] Standardize age for comparability because it is numeric while the rest of the variables are binary
#scaler = StandardScaler()
#data['age_scaled'] = scaler.fit_transform(data[['age']])

# Group age into bins
data['age_group'] = pd.qcut(data['age'], q = 5, duplicates = 'drop')

# One-hot encode the bins
age_encode = pd.get_dummies(data['age_group'], prefix = 'age', drop_first = True)

# Merge age_encode into main dataframe
data = pd.concat([data, age_encode], axis = 1)

In [ ]:
# Define predictors and outcome
predictors = [
    'age_encode',
    'gender', 
    'polyuria', 
    'polydipsia', 
    'sudden_weight_loss', 
    'weakness', 
    'polyphagia', 
    'genital_thrush', 
    'visual_blurring', 
    'itching', 
    'irritability', 
    'delayed_healing', 
    'partial_paresis', 
    'muscle_stiffness', 
    'alopecia', 
    'obesity'
] + list(age_encode.columns)

X = data[predictors]
y = data['diabetes']

KeyError: "['age_encode'] not in index"

In [ ]:
# Add constant and fit logistic regression
X = sm.add_constant(X)
model = sm.Logit(y, X)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.165053
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:               diabetes   No. Observations:                  520
Model:                          Logit   Df Residuals:                      503
Method:                           MLE   Df Model:                           16
Date:                Wed, 12 Nov 2025   Pseudo R-squ.:                  0.7523
Time:                        18:49:48   Log-Likelihood:                -85.827
converged:                       True   LL-Null:                       -346.46
Covariance Type:            nonrobust   LLR p-value:                1.067e-100
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.2890      0.542      0.533      0.594      -0.774       1.352
age_

In [ ]:
# Calculating odds ratios
odds_ratios = pd.DataFrame({
    "Variable": result.params.index,
    "Odds_Ratio": np.exp(result.params),
    "p-value": result.pvalues
})

print(odds_ratios.sort_values("p-value"))

                              Variable  Odds_Ratio       p-value
gender                          gender    0.012892  3.491487e-13
polyuria                      polyuria   84.735923  3.077802e-10
polydipsia                  polydipsia  159.244575  9.522435e-10
itching                        itching    0.060632  3.088698e-05
irritability              irritability   10.388796  7.376964e-05
genital_thrush          genital_thrush    6.447231  7.566817e-04
polyphagia                  polyphagia    3.299485  2.525063e-02
partial_paresis        partial_paresis    3.187711  2.717698e-02
age_scaled                  age_scaled    0.537313  4.365732e-02
weakness                      weakness    2.263847  1.279860e-01
visual_blurring        visual_blurring    2.498960  1.596007e-01
muscle_stiffness      muscle_stiffness    0.482507  2.091021e-01
delayed_healing        delayed_healing    0.675951  4.764573e-01
const                            const    1.335134  5.940674e-01
obesity                  

The logistic regression shows that polydipsia, polyuria, and irritability have strong positive association with diabetes once other symptoms are accounted for.

In [ ]:
# Logistic regression with gender x polyuria interaction
model_interaction = smf.logit(    
    formula = 'diabetes ~ gender + polyuria + gender:polyuria',
    data = data
).fit()

# Display regression summary
print(model_interaction.summary())

# Convert coefficients to odds ratio
odds_ratios = np.exp(model_interaction.params)
print("\nOdds Ratios:")
print(odds_ratios)

         Current function value: 0.335250
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:               diabetes   No. Observations:                  520
Model:                          Logit   Df Residuals:                      516
Method:                           MLE   Df Model:                            3
Date:                Wed, 12 Nov 2025   Pseudo R-squ.:                  0.4968
Time:                        18:51:40   Log-Likelihood:                -174.33
converged:                      False   LL-Null:                       -346.46
Covariance Type:            nonrobust   LLR p-value:                 2.597e-74
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.8398      0.275      3.059      0.002       0.302       1.378
gender             -2.4552      0.334     -7.347  

c:\Users\krist\miniconda3\envs\dsi_participant\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


The p-value for the gender x polyuria interaction term is not significant. The relationship between polyuria and diabetes doesn't differ by gender meaningfully. The odds ratio (1.363e-05) is very close to 1, indicating small effect difference between genders.

In [ ]:
# Logistic regression with gender x polydipsia interaction
model_interaction = smf.logit(    
    formula = 'diabetes ~ gender + polydipsia + gender:polydipsia',
    data = data
).fit()

# Display regression summary
print(model_interaction.summary())

# Convert coefficients to odds ratio
odds_ratios = np.exp(model_interaction.params)
print("\nOdds Ratios:")
print(odds_ratios)

         Current function value: 0.351138
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:               diabetes   No. Observations:                  520
Model:                          Logit   Df Residuals:                      516
Method:                           MLE   Df Model:                            3
Date:                Wed, 12 Nov 2025   Pseudo R-squ.:                  0.4730
Time:                        18:59:25   Log-Likelihood:                -182.59
converged:                      False   LL-Null:                       -346.46
Covariance Type:            nonrobust   LLR p-value:                 9.814e-71
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.9268      0.271      3.419      0.001       0.396       1.458
gender               -2.2299      0.317     

c:\Users\krist\miniconda3\envs\dsi_participant\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


The p-value for the gender x polydipsia interaction term is not significant. The relationship between polydipsia and diabetes doesn't differ by gender meaningfully. The odds ratio (1.073e-08) is very close to 1, indicating small effect difference between genders.

In [ ]:
# Logistic regression with gender x irritability interaction
model_interaction = smf.logit(    
    formula = 'diabetes ~ gender + irritability + gender:irritability',
    data = data
).fit()

# Display regression summary
print(model_interaction.summary())

# Convert coefficients to odds ratio
odds_ratios = np.exp(model_interaction.params)
print("\nOdds Ratios:")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.494014
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:               diabetes   No. Observations:                  520
Model:                          Logit   Df Residuals:                      516
Method:                           MLE   Df Model:                            3
Date:                Wed, 12 Nov 2025   Pseudo R-squ.:                  0.2585
Time:                        20:10:24   Log-Likelihood:                -256.89
converged:                       True   LL-Null:                       -346.46
Covariance Type:            nonrobust   LLR p-value:                 1.343e-38
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               1.9459      0.252      7.723      0.000       1.452       2.440
ge

The p-value for the gender x irritability interaction term is not significant. The relationship between irritability and diabetes doesn't differ by gender meaningfully. The odds ratio (1.236) is very close to 1, indicating small effect difference between genders.

In [ ]:
# Remove gender from predictors
predictors_no_gender = [
    'polyuria', 
    'polydipsia', 
    'sudden_weight_loss', 
    'weakness', 
    'polyphagia', 
    'genital_thrush', 
    'visual_blurring', 
    'itching', 
    'irritability', 
    'delayed_healing', 
    'partial_paresis', 
    'muscle_stiffness', 
    'alopecia', 
    'obesity'
] + list(age_encode.columns)

In [ ]:
# Stratify by gender
males = data[data["gender"] == 1]
females = data[data["gender"] == 0]

# Fit logistic regression for males
model_male = sm.Logit(males["diabetes"], sm.add_constant(males[predictors_no_gender])).fit()

# Convert to odds ratios
odds_ratios = np.exp(model_male.params)

# 95% confidence intervals
conf = np.exp(model_male.conf_int())
conf.columns = ['2.5%', '97.5%']

# Extract p-values for summary table
p_values = model_male.pvalues

# Summary table
results_male = pd.concat([odds_ratios, conf, p_values], axis=1)
results_male.columns = ['OR', '2.5%', '97.5%', 'p-value']

# Sort by p-value, ascending
results_male_sorted = results_male.sort_values(by = 'p-value', ascending = True)

print(results_male_sorted)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

Polydipsia, polyuria, and irritability are variables that are both statistically and clinically meaningful for males in predicting diabetes.

In [ ]:
# Upsample females as current dataset has 328 males and 192 females [double-check if this is best way to go about this]
#sample = females.sample(n = 328, replace = True)

# Bootstrap female samples because it is a much smaller pool than males
bootstrap_females = []

for i in range(2000):
    sample = females.sample(n = 328, replace = True)
    sample = sample.assign(replicate = i)
    bootstrap_females.append(sample)

samples = pd.concat(bootstrap_females)
samples

,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity,diabetes,age_scaled,replicate
186,90,0,0,1,1,0,0,1,1,1,0,0,0,1,1,0,1,3.457325,0
362,28,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,-1.649853,0
197,61,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,1.068484,0
77,55,0,1,1,1,0,1,0,0,1,0,1,1,0,0,0,1,0.574241,0
252,39,0,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,-0.743741,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,39,0,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,-0.743741,1999
62,55,0,1,1,0,1,1,0,1,1,0,1,1,1,0,0,1,0.574241,1999
64,45,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,-0.249498,1999
301,47,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,1,-0.084750,1999


In [ ]:
for col in predictors_no_gender:
    tab = pd.crosstab(one_sample[col], one_sample["diabetes"])
    if (tab == 0).any().any():
        print(col)
        print(tab)
        print("Perfect or quasi separation detected.\n")


polyuria
diabetes   0    1
polyuria         
0         30   79
1          0  219
Perfect or quasi separation detected.

polydipsia
diabetes     0    1
polydipsia         
0           30   94
1            0  204
Perfect or quasi separation detected.



KeyError: 'age_(37.0, 44.0]'

In [ ]:
# Take random subset of 328 rows from samples for fit
one_sample = samples.sample(n = 328, replace = False)

# Fit logistic regression for females
model_female = sm.Logit(one_sample["diabetes"], sm.add_constant(one_sample[predictors_no_gender])).fit()

# Convert to odds ratios
odds_ratios = np.exp(model_female.params)

# 95% confidence intervals
conf = np.exp(model_female.conf_int())
conf.columns = ['2.5%', '97.5%']

# Extract p-values for summary table
p_values = model_female.pvalues

# Summary table
results_female = pd.concat([odds_ratios, conf, p_values], axis=1)
results_female.columns = ['OR', '2.5%', '97.5%', 'p-value']

# Sort by p-value, ascending
results_female_sorted = results_female.sort_values(by = 'p-value', ascending = True)

print(results_female_sorted)

KeyError: "['age_(37.0, 44.0]', 'age_(44.0, 50.0]', 'age_(50.0, 58.0]', 'age_(58.0, 90.0]'] not in index"